# Parte 1: Utilidade

## Importar os pacotes necessários

In [2]:
# importar os pacotes necessários para fazer análise
import pandas as pd, numpy as np
import os, re
import requests
import zipfile

## Definir funções para processar os dados

In [3]:
# definir função que baixa os dados
def baixar_tse():
    URL = 'http://agencia.tse.jus.br/estatistica/sead/odsele/motivo_cassacao/motivo_cassacao_2016.zip'
    r = requests.get(URL)
    with open('../dados/motivo_cassacao2016.zip', 'wb') as arquivo:
        arquivo.write(r.content)
    return 'Motivo(s) para Indeferimento 2016 baixado(s) com sucesso.'

# definir função de descomprime arquivo do tse
def unzip_tse():
    with zipfile.ZipFile('../dados/motivo_cassacao2016.zip', 'r') as zip:
        zip.extractall('../dados')
    return 'Arquivo(s) descomprimido(s) com sucesso.'

# definir função para juntar os dados do tse
def juntar_tse():
    regex = re.compile(r'motivo.*csv$')
    juntar_tse.kwargs = {
        'engine': 'python', 'encoding': 'latin1', 'sep':';', 'quoting': 1,
        'dtype': str
    }
    arquivos = os.listdir('../dados')
    arquivos = list(filter(regex.search, arquivos))
    arquivos = [
        os.path.join(os.path.realpath('../dados'), arquivo) \
        for arquivo in arquivos
    ]
    dados = pd.concat(
        [pd.read_csv(arquivo, **juntar_tse.kwargs) for arquivo in arquivos],
        ignore_index=True
    )
    dados = dados[dados['DS_MOTIVO_CASSACAO'].notna()]
    print('Arquivo(s) juntado(s) com sucesso.')
    return dados

## Executar funções

In [4]:
# executar função
baixar_tse()

'Motivo(s) para Indeferimento 2016 baixado(s) com sucesso.'

In [5]:
# executar função
unzip_tse()

'Arquivo(s) descomprimido(s) com sucesso.'

In [6]:
# executar função e salvar o resultado no banco de dados chamado "candidaturas"
candidaturas = juntar_tse()

Arquivo(s) juntado(s) com sucesso.


## Exibir dados preliminares

In [7]:
# mostrar os dados
candidaturas.sample(10)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,SG_UF,SG_UE,NM_UE,SQ_CANDIDATO,DS_MOTIVO_CASSACAO
12545,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,SP,61573,ARAÇOIABA DA SERRA,250000013929,Abuso de poder (LC 64/90)
12162,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,SP,71072,SÃO PAULO,250000093980,Ausência de requisito de registro
12988,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,SP,62910,CAMPINAS,250000031045,Ausência de requisito de registro
1237,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,AM,02593,MARAÃ,40000001820,Ausência de requisito de registro
5005,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,PE,24236,GOIANA,170000015671,Indeferimento de partido ou coligação.
3623,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,BA,33634,BARREIRAS,50000017648,Ausência de requisito de registro
18004,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,MT,91170,NOBRES,110000004691,Ausência de requisito de registro
8127,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,RJ,60011,RIO DE JANEIRO,190000011863,Ausência de requisito de registro
15199,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,MS,90859,IGUATEMI,120000005804,Ausência de requisito de registro
4847,23/01/2020,08:45:56,2016,2,Eleição Ordinária,220,Eleições Municipais 2016,PE,25496,SANTA CRUZ DO CAPIBARIBE,170000015088,Ausência de requisito de registro


In [8]:
# verificar os motivos para cassação
list(candidaturas['DS_MOTIVO_CASSACAO'].unique())

['Ausência de requisito de registro ',
 'Indeferimento de partido ou coligação.',
 'Compra de voto (Lei 9.504/97).',
 'Ficha limpa (LC 64/90)',
 'Abuso de poder (LC 64/90)',
 'Gasto ilícito de recursos (Lei 9.504/97).',
 'Impugnação',
 'Conduta vedada (Lei 9.504/97).']

## Focar no Paraná

In [9]:
# focar em 10 candidatos no paraná
amostra_pr = candidaturas[candidaturas['SG_UF'] == 'PR'].sample(10, random_state=67)

In [10]:
# mostrar quais informações nós obtivemos
amostra_pr.columns.to_list()

['DT_GERACAO',
 'HH_GERACAO',
 'ANO_ELEICAO',
 'CD_TIPO_ELEICAO',
 'NM_TIPO_ELEICAO',
 'CD_ELEICAO',
 'DS_ELEICAO',
 'SG_UF',
 'SG_UE',
 'NM_UE',
 'SQ_CANDIDATO',
 'DS_MOTIVO_CASSACAO']

In [11]:
# juntar os motivos de cassação com outros dados dos candidatos?
candidatos_pr = pd.read_csv('../dados/consulta_cand_2016_PR.csv', **juntar_tse.kwargs)

In [12]:
# puxar os números do protocolo da candidatura
amostra_pr = amostra_pr.merge(candidatos_pr, on='SQ_CANDIDATO')

In [13]:
# disponibilizar na tela os números dos protocolos de candidatura
# amostra_pr.columns
amostra_pr[['NM_CANDIDATO', 'NM_UE_x', 'DS_CARGO', 'NR_PROTOCOLO_CANDIDATURA']]

,NM_CANDIDATO,NM_UE_x,DS_CARGO,NR_PROTOCOLO_CANDIDATURA
0,MAURICIO QUIMENTON COSTA,LONDRINA,VEREADOR,1194392016
1,VERA ILSA BALDUINO DA SILVA,NOVA OLÍMPIA,VEREADOR,1225872016
2,FERNANDO DANIEL SCHMIDT,SÃO JORGE DO PATROCÍNIO,VEREADOR,1013362016
3,LUCIANO DA SILVA,APUCARANA,VEREADOR,1134682016
4,LEANDRO PIRES DA SILVA,COLORADO,VEREADOR,1065522016
5,JOSE MARIA DA SILVA,ARAPONGAS,VEREADOR,1182182016
6,JOSINEL PEZINI DOS SANTOS,PEABIRU,VEREADOR,1247862016
7,MARIA NEUSA DE LIMA,GUARATUBA,VEREADOR,1969862016
8,VINICIUS RIBEIRO RAMOS,TELÊMACO BORBA,VEREADOR,1211272016
9,PAULA TERESINHA REVERS,QUEDAS DO IGUAÇU,VEREADOR,1151382016


In [14]:
# criar link para raspar documentos
links = [
    f'http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?' \
    f'nprot={nprot}&comboTribunal=pr' \
    for nprot in amostra_pr['NR_PROTOCOLO_CANDIDATURA'].to_list()
]

# disponibilizar os links para a gente
for link in links: 
    print(link)

http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1194392016&comboTribunal=pr
http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1225872016&comboTribunal=pr
http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1013362016&comboTribunal=pr
http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1134682016&comboTribunal=pr
http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1065522016&comboTribunal=pr
http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1182182016&comboTribunal=pr
http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1247862016&comboTribunal=pr
http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1969862016&comboTribunal=pr
http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1211272016&comboTribunal=pr
http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?nprot=1151382016&comboTribunal=pr


## Baixar e processar os dados

In [15]:
# usar módulo externo

In [16]:
# importar meu pacote de análise
from tse import parser

In [17]:
# construir path para os arquivos
decisões = [f'../dados/decisão_{i}.html' for i in range(0, 10)]

In [18]:
# definir função para transformar os dados do sumário em tabela
def tabelar_sumário(decisão, transpor=True):
    sumário = pd.DataFrame(parser(decisão).parse_summary())
    if transpor: 
        sumário = sumário.T
    return sumário

In [19]:
# produzir tabela
tabelar_sumário(decisões[1])

,0
case,Nº 0000299-72.2016.6.16.0127 - REGISTRO DE CAN...
town,NOVA OLÍMPIA-PRN.° Origem:
prot,1225872016 - 15/08/2016 18:13
claimants,COLIGAÇÃO GOVERNANDO COM O POVO (PMDB / PSC / ...
defendant,"VERA ILSA BALDUINO DA SILVA, CARGO VEREADOR, N..."
judge,FERNANDA BATISTA DORNELLES
subject,DIREITO ELEITORAL - Eleições - Candidatos - Re...
district,127ZE-127 ZONA ELEITORAL
stage,09/02/2017 12:21-Arquivado na seção


In [20]:
# criar banco de dados
sumários = pd.concat([tabelar_sumário(decisão, False) for decisão in decisões], ignore_index=True)
sumários

,case,town,prot,claimants,defendant,judge,subject,district,stage
0,Nº 0000304-61.2016.6.16.0041 - REGISTRO DE CAN...,LONDRINA-PRN.° Origem:,1194392016 - 15/08/2016 17:09,COLIGAÇÃO LONDRINA PRA FRENTE (PP / PTB),"MAURICIO QUIMENTON COSTA, CARGO VEREADOR, Nº: ...",MATHEUS ORLANDI MENDES,DIREITO ELEITORAL - Eleições - Candidatos - Re...,041ZE-041 ZONA ELEITORAL,25/09/2016 15:56-Arquivado na seção
1,Nº 0000299-72.2016.6.16.0127 - REGISTRO DE CAN...,NOVA OLÍMPIA-PRN.° Origem:,1225872016 - 15/08/2016 18:13,COLIGAÇÃO GOVERNANDO COM O POVO (PMDB / PSC / ...,"VERA ILSA BALDUINO DA SILVA, CARGO VEREADOR, N...",FERNANDA BATISTA DORNELLES,DIREITO ELEITORAL - Eleições - Candidatos - Re...,127ZE-127 ZONA ELEITORAL,09/02/2017 12:21-Arquivado na seção
2,Nº 0000050-36.2016.6.16.0123 - REGISTRO DE CAN...,SÃO JORGE DO PATROCÍNIO-PRN.° Origem:,1013362016 - 12/08/2016 18:24,"COLIGAÇÃO PSD, PP, PTN (PSD / PP / PTN)","FERNANDO DANIEL SCHMITD, CARGO VEREADOR, Nº: 5...",GUILHERME ARANDA CASTRO DOS SANTOS,DIREITO ELEITORAL - Eleições - Candidatos - Re...,123ZE-123 ZONA ELEITORAL,23/11/2016 12:10-Arquivado na seção
3,Nº 0000161-14.2016.6.16.0028 - REGISTRO DE CAN...,APUCARANA-PRN.° Origem:,1134682016 - 15/08/2016 14:58,"COLIGAÇÃO PTB-PMDB-PPS (PTB / PMDB / PPS), COL...","JUÍZO ELEITORAL DA 28ª ZONA, TRE-PR",RENATA BOLZAN JAURIS,DIREITO ELEITORAL - Eleições - Candidatos - Re...,028ZE-028 ZONA ELEITORAL,20/03/2017 16:41-Arquivado na seção
4,Nº 0000186-20.2016.6.16.0095 - REGISTRO DE CAN...,COLORADO-PRN.° Origem:,1065522016 - 15/08/2016 12:32,COLIGAÇÃO JUNTOS POR COLORADO (PPL / PT DO B /...,"LEANDRO PIRES DA SILVA, CARGO VEREADOR, Nº: 54000",LUCIANA PAULA KULEVICZ,DIREITO ELEITORAL - Eleições - Candidatos - Re...,095ZE-095 ZONA ELEITORAL,07/03/2017 17:30-Arquivado na seção
5,Nº 0000280-70.2016.6.16.0061 - REGISTRO DE CAN...,ARAPONGAS-PRN.° Origem:,1182182016 - 15/08/2016 16:37,COLIGAÇÃO PP E SD (PP / SD),"JOSE MARIA DA SILVA, CARGO VEREADOR, Nº: 11611",RENATA MARIA FERNANDES SASSI FANTIN,DIREITO ELEITORAL - Eleições - Candidatos - Re...,061ZE-061 ZONA ELEITORAL,14/03/2017 17:44-Arquivado na seção
6,Nº 0000180-76.2016.6.16.0074 - REGISTRO DE CAN...,PEABIRU-PRN.° Origem:,1247862016 - 15/08/2016 18:54,COLIGAÇÃO PSB - PDT - PEN - PT DO B - PSC - PR...,"JOSINEL PEZINI DOS SANTOS, CARGO VEREADOR, Nº:...",PAULO EDUARDO MARQUES PEQUITO,DIREITO ELEITORAL - Eleições - Candidatos - Re...,074ZE-074 ZONA ELEITORAL,19/05/2020 18:50-Certidão
7,Nº 0000259-85.2016.6.16.0161 - REGISTRO DE CAN...,GUARATUBA-PRN.° Origem:,1969862016 - 12/09/2016 17:54,COLIGAÇÃO GUARATUBA MAIS HUMANA (PMDB / PT),"MARIA NEUSA DE LIMA, CARGO VEREADOR, Nº: 13321",GIOVANNA DE SÁ RECHIA,DIREITO ELEITORAL - Eleições - Candidatos - Re...,161ZE-161 ZONA ELEITORAL,13/04/2017 16:35-Arquivado na seção
8,Nº 0000271-55.2016.6.16.0111 - REGISTRO DE CAN...,TELÊMACO BORBA-PRN.° Origem:,1211272016 - 15/08/2016 17:44,COLIGAÇÃO JUNTOS POR TELÊMACO BORBA (PMB / PSD...,"VINICIUS RIBEIRO RAMOS, CARGO VEREADOR, Nº: 45000",MARCELO FURLANETTO DA FONSECA,DIREITO ELEITORAL - Eleições - Candidatos - Re...,111ZE-111 ZONA ELEITORAL,31/03/2017 15:39-Arquivado na seção
9,Nº 0000186-10.2016.6.16.0163 - REGISTRO DE CAN...,QUEDAS DO IGUAÇU-PRN.° Origem:,1151382016 - 15/08/2016 15:27,COLIGAÇÃO A FORÇA QUE VEM DO POVO (PMDB / PDT ...,"PAULA TERESINHA REVERS, CARGO VEREADOR, Nº: 27555",PAULA CHEDID MAGALHÃES,DIREITO ELEITORAL - Eleições - Candidatos - Re...,163ZE-163 ZONA ELEITORAL,14/03/2017 16:46-Arquivado na seção


In [21]:
# onde estão estes processos? nas seguintes zonas eleitorais
sumários['district']

0    041ZE-041 ZONA ELEITORAL
1    127ZE-127 ZONA ELEITORAL
2    123ZE-123 ZONA ELEITORAL
3    028ZE-028 ZONA ELEITORAL
4    095ZE-095 ZONA ELEITORAL
5    061ZE-061 ZONA ELEITORAL
6    074ZE-074 ZONA ELEITORAL
7    161ZE-161 ZONA ELEITORAL
8    111ZE-111 ZONA ELEITORAL
9    163ZE-163 ZONA ELEITORAL
Name: district, dtype: object

## Parte 3: Processamento de Linguagem Natural (NLP)

In [ ]:
# importar pacotes
import spacy
from bs4 import BeautifulSoup

# criar ferramentas de análise de texto
nlp_en = spacy.load('en_core_web_sm')
nlp_pt = spacy.load('pt_core_news_sm')

### Exemplo Genérico

In [ ]:
# definir função que baixa uma reportagem de jornal
def baixar_reportagem():
    URL = 'https://www.nytimes.com/2020/05/12/well/family/coronavirus-children-covid-19.html'
    r = requests.get(URL)
    soup = BeautifulSoup(r.content)
    text = soup.find('section', attrs={'name': 'articleBody'}).get_text()
    text = re.sub(r'(”|“)(?![A-z])', r'\1 ', text)
    text = re.sub(r' +', ' ', text)
    return text

In [ ]:
# baixar reportagem do NYTimes
baixar_reportagem()

In [ ]:
# salvar texto
texto = baixar_reportagem()

In [ ]:
# processar o texto
doc_en = nlp_en(texto)

In [ ]:
# contar o número de frases
frases = list(doc_en.sents)
len(frases)

In [ ]:
# mostrar uma frase qualquer
frases[17]

In [ ]:
# definir função para investigar o conteúdo de uma frase
def investigar_frase(frase):
    resultado = [{'token': ent.text, 'tipo': ent.label_ , 'descrição': spacy.explain(ent.label_)} for ent in frase.ents]
    return pd.DataFrame(resultado)

In [ ]:
# testar reconhecimento do texto
print(f'{frases[17]}\n\n{investigar_frase(frases[17])}')

In [ ]:
# testar reconhecimento do texto
print(f'{frases[38]}\n\n{investigar_frase(frases[38])}')

### Exemplo Real

In [ ]:
# abrir uma coletânea de decisões judiciais dentre todas
decisão = pd.DataFrame(parser(decisões[1]).parse_details())
decisão

In [ ]:
# extrair texto da decisão
texto = decisão.loc[3, 'sbody']
texto

### Mostrar as entidades reconhecidas pelo software nos autos

In [ ]:
# processar o texto 
doc_pt = nlp_pt(texto)

In [ ]:
# transformar em frases
frases = list(doc_pt.sents)

In [ ]:
# imprimir as entidades reconhecidas pela nossa ferramenta
pd.concat([pd.DataFrame(investigar_frase(frase)) for frase in frases])

### Análise Sintática

In [ ]:
# fazer análise sintática do texto
frase = str(frases[-3])
frase = nlp_pt(frase)

In [ ]:
# interpretar a frase mais detalhadamente
pd.DataFrame([
    {'token': token.text, 'classe gramatical': token.pos_, 'descrição': spacy.explain(token.pos_)}
    for token in frase
])

In [ ]:
spacy.displacy.render(frase, jupyter=True, style='dep', options={'compact': True})

### Destacar as entidades no texto

In [ ]:
# mostrar as entidades na frase
colors = {'ORG': 'linear-gradient(90deg, #aa9cfc, #fc9ce7)'}
options = {'ents': 'ORG', 'colors': colors}
spacy.displacy.render(frase, jupyter=True, style='ent', options=options)